In [1]:
using Pkg; Pkg.update

using ParallelTemperingMonteCarlo,BenchmarkTools,StaticArrays

Looking into the implementation of the EAM. A semi-two-body potential 

In [2]:
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
[0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
[0.1811648830,        0.5575692066,        0.2931316748],
[0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
[0.4742970144,        0.3445967202,       -0.2931309590],
[0.4742970205,       -0.3445967231,       -0.2931309559],
[0.0000007214,       -0.0000000073,       -0.6554619185]]
ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]

nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13
pos_cu55 = copperconstant*ico_55
AtoBohr = 1.8897259886
bc_cu13 = SphericalBC(radius=8*AtoBohr)
start_config = Config(pos_cu13, bc_cu13)

Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-3.35737368666e-8, -3.01478453496e-8, -2.2610884012200003e-8], [-5.0100237544608e-6, -9.5924962476e-9, 4.491082807389623], [1.2413025402138254, -3.8203432492477134, 2.0084746703815606], [-3.2497803034902475, -2.3611020206489175, 2.00846968707976], [-3.249780345286124, 2.3611019212980637, 2.0084696658392325], [1.241302471695995, 3.8203432300627207, 2.0084746361226458], [4.016944341017041, 1.50739226748e-8, 2.008477729017507], [-1.2413025388434689, -3.8203432896732337, -2.0084746806592353], [-4.016944408164515, -7.468443507060001e-8, -2.008477773554097], [-1.2413026073612992, 3.8203431896372013, -2.0084747149181505], [3.2497802363427737, 2.361101961038405, -2.00846973161635], [3.2497802781386502, -2.361101980908576, -2.0084697103758224], [4.9428762807276005e-6, -5.0018016148200005e-8, -4.491082852611391]], SphericalBC{Float64}(228.54811596737582))

Generating the necessary dist2_matrix

In [3]:
dis2mat = [distance2(a,b) for a in pos_cu13, b in pos_cu13]
dist2mat_55 = [distance2(a,b) for a in pos_cu55, b in pos_cu55]

55×55 Matrix{Float64}:
  0.0      20.939    20.939    20.939   …   62.2806   62.2806   82.1728
 20.939     0.0      23.1496   23.1496     144.657   144.657   186.072
 20.939    23.1496    0.0      23.1496     121.19     83.2196  140.213
 20.939    23.1496   23.1496    0.0        144.657   121.19    140.213
 20.939    23.1496   60.6064   23.1496     121.19    144.657   140.213
 20.939    23.1496   60.6064   60.6064  …   83.2196  121.19    140.213
 20.939    23.1496   23.1496   60.6064      83.2196   83.2196  140.213
 20.939    60.6064   23.1496   23.1496      83.2196   45.249    66.0106
 20.939    60.6064   60.6064   23.1496      83.2196   83.2196   66.0106
 20.939    60.6064   83.756    60.6064      45.249    83.2196   66.0106
  ⋮                                     ⋱                      
 82.1728  140.213   140.213   186.072       22.7447   69.2332   90.848
 62.2806  121.19     83.2196  144.657       23.7891   23.7891   69.2332
 82.1728  140.213    66.0106  140.213       69.2332   22

The mathematical form of the EAM is:

$$
E = \frac{1}{2}\sum_{i,j :i \neq j}\epsilon\left(\frac{a}{r_{ij}}\right)^n - \epsilon C \sum_{i}\left[\sum_{j\neq i} \left(\frac{a}{r_{ij}}\right)^m \right]^{1/2}
$$

We have two terms, one similar to the LJ repulsion term, one a density term $F(\rho _i)$.

We can consider that $\frac{\epsilon a^n}{2}$ and $\epsilon C a^{m/2} $ can be taken out of the sum. Thus a single sum represents contracting a vector in the dist matrix. 

In [4]:
# struct EmbeddedAtomPotential <: AbstractDimerPotential
#     n::Float64
#     m::Float64
#     ean::Float64
#     eCam::Float64
# end
# function EmbeddedAtomPotential(n,m,ϵ,C,a)
#     epsan = ϵ * a^n / 2
#     epsCam = ϵ * C * a^(m/2)
#     return EmbeddedAtomPotential(n,m,epsan,epsCam)
# end

In [5]:
evtohartree = 0.0367493

n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561


copperpot = EmbeddedAtomPotential(n,m,ϵ,C,a)

suttonchenpot = EmbeddedAtomPotential(9.0,6.0,0.0126*evtohartree,39.432,0.3612*nmtobohr)

copperpot2 = EmbeddedAtomPotential(9.05 , 5.005 , 0.0225*evtohartree,33.17,0.327*nmtobohr)

EmbeddedAtomPotential(9.05, 5.005, 5949.446479587037, 2.6153101861477057)

In [6]:
invrexp(r2,expterm) = ifelse(r2 != 0 , 1/sqrt(r2)^expterm, 0. )
calcvec(distvec,expterm) = sum(invrexp.(distvec,Ref(expterm)))

function energyatomi(distvec,pot::EmbeddedAtomPotential) 
    Fterm = pot.eCam*sqrt(calcvec(distvec,pot.m))
    phiterm = pot.ean*calcvec(distvec,pot.n)
    return phiterm - Fterm
end
function energytotal(distmatrix,pot::EmbeddedAtomPotential)
    total_energy = 0.
    for distrow in eachrow(distmatrix)
        total_energy += energyatomi(distrow,pot)
    end
    return total_energy 
end
E_vec = [energyatomi(distrow,copperpot) for distrow in eachrow(dis2mat)]
sum(E_vec)

E_vec2 = [energyatomi(distrow,suttonchenpot) for distrow in eachrow(dis2mat)]
sum(E_vec2)

-1.3002351409450545

In [7]:
energytotal(dis2mat,suttonchenpot)

-1.3002351409450545

In [8]:
@benchmark energytotal($dist2mat_55,$suttonchenpot)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  34.349 μs …  3.476 ms  ┊ GC (min … max): 0.00% … 98.33%
 Time  (median):     45.094 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   44.781 μs ± 66.857 μs  ┊ GC (mean ± σ):  2.95% ±  1.96%

     █▂▃               ▃▄▅▂▁                                   
  █▇▇███▅▃▂▂▂▂▁▁▁▁▁▂▅▇██████▆▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▁▁▂▁▁▁▁▁▁ ▃
  34.3 μs         Histogram: frequency by time        64.6 μs <

 Memory estimate: 53.28 KiB, allocs estimate: 110.

# An alternative approach

Considering that the $\rho_i$ term depends on the square of sums, we can't simply update the total energy by considering the change to a single $r_{ij}$ on the global energy.

Rather than calculating $E_i$ and being unable to _separate_ the terms, we can store the $\phi_i,\rho_i$ terms. This may result in slower calculation of the global terms, but it may speed up the calculation of $E_{new}$ at each step.

In [9]:
function invrexp(r2,n,m)
    if r2 != 0.
        r_term = 1/sqrt(r2)
        return r_term^n , r_term^m
    else
        return 0. , 0.
    end    
end

function calc_components(eatomvec,distancevec,n,m)
    for dist in distancevec
        eatomvec .+= invrexp(dist,n,m)
    end
    return eatomvec
end
function calc_all_components(dist2_mat,n,m)
    n_atoms = size(dist2_mat)[1]
    component_vector = zeros(n_atoms,2)
    for row_index in 1:n_atoms
        component_vector[row_index,:] = calc_components(component_vector[row_index,:],dist2_mat[row_index,:],n,m)
    end
    return component_vector
end
function calc_energies_from_components(component_vector,ean,ecam)
    return sum(ean.*component_vector[:,1] .- ecam*sqrt.(component_vector[:,2]))
end
function calc_total_energy(dist2_mat,pot::EmbeddedAtomPotential)
    componentvec = calc_all_components(dist2_mat,pot.n,pot.m)
    return componentvec,calc_energies_from_components(componentvec,pot.ean,pot.eCam) 
end

calc_total_energy (generic function with 1 method)

In [10]:
phirho,energytot = calc_total_energy(dist2mat_55,suttonchenpot)

([1.3923704520825358e-5 0.0014529219103246252; 1.1152248517093746e-5 0.0012372868606771957; … ; 6.176322918802852e-6 0.0007295761752523578; 5.340590516223844e-6 0.0005944969082196186], -6.119246949010012)

In [11]:
@benchmark calc_total_energy($dist2mat_55,$copperpot)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  116.563 μs …  4.957 ms  ┊ GC (min … max): 0.00% … 96.66%
 Time  (median):     126.348 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   128.497 μs ± 83.355 μs  ┊ GC (mean ± σ):  1.11% ±  1.68%

      ▁   █▃   ▁   ▂                                            
  ▃▁▁▁█▃▂▂██▄▄▆█▅▅▇██▆▅▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  117 μs          Histogram: frequency by time          152 μs <

 Memory estimate: 34.34 KiB, allocs estimate: 116.

In [12]:
function update_components(componentvec,atomindex,old_r2_vec,new_r2_vec,n,m)
    new_component_vec= copy(componentvec)

    for j_index in eachindex(new_r2_vec)
        j_term = invrexp(new_r2_vec[j_index],n,m) .- invrexp(old_r2_vec[j_index],n,m)
        new_component_vec[j_index,:] .+= j_term 
        new_component_vec[atomindex,:] .+= j_term 
    end
    return new_component_vec
end

update_components (generic function with 1 method)

In [13]:
testindex,test_disp = 3,[0.5,0.5,-0.5]
test_pos = pos_cu55[testindex] + test_disp

3-element Vector{Float64}:
  1.764752985534391
 -3.3925157884546686
  1.5464230925690106

In [14]:
newdisvector = [distance2(test_pos,b) for b in pos_cu55]
newdisvector[testindex] = 0.

testenergyd2mat = copy(dist2mat_55)
testenergyd2mat[3,:] = newdisvector
testenergyd2mat[:,3] = newdisvector

newphirho,newentot = calc_total_energy(testenergyd2mat,suttonchenpot)

new_component_vector = update_components(phirho,3,dist2mat_55[3,:],newdisvector,9.,6.)
enew = calc_energies_from_components(new_component_vector,suttonchenpot.ean,suttonchenpot.eCam)

-6.0788077775169675

In [15]:
function calc_new_energy(componentvec,atomindex,old_r2,new_r2,pot::EmbeddedAtomPotential)
    new_component_vector = update_components(componentvec,atomindex,old_r2,new_r2,pot.n,pot.m)
    newenergy = calc_energies_from_components(new_component_vector,pot.ean,pot.eCam)

    return new_component_vector , newenergy 
end

calc_new_energy (generic function with 1 method)

In [16]:
calc_new_energy(phirho,3,dist2mat_55[3,:],newdisvector,suttonchenpot)[2]

-6.0788077775169675

In [17]:
@benchmark calc_new_energy($phirho,$3,$dist2mat_55[3,:],$newdisvector,$suttonchenpot)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.934 μs … 602.042 μs  ┊ GC (min … max):  0.00% … 98.63%
 Time  (median):     3.551 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.974 μs ±  20.035 μs  ┊ GC (mean ± σ):  13.87% ±  3.42%

    ██                                                         
  ▂▆███▅▃▃▃▂▂▂▂▂▂▂▁▁▁▁▂▂▁▁▁▁▂▃▄▄▅▅▄▄▃▃▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.93 μs         Histogram: frequency by time        7.67 μs <

 Memory estimate: 12.48 KiB, allocs estimate: 117.

# Finishing up

With the majority of the functions complete, it's just a matter of including them in the main code. Specifically how this is done is never the most fun challenge, but something to work on nevertheless.

In [18]:
get_energy(dist2mat_55,suttonchenpot)

([1.3923704520825358e-5 0.0014529219103246252; 1.1152248517093746e-5 0.0012372868606771957; … ; 6.176322918802852e-6 0.0007295761752523578; 5.340590516223844e-6 0.0005944969082196186], -6.119246949010012)

# Incorporating this  into MCRun

With the energy functions complete, it's time to migrate this to mc_cycles. Because we have the additional output, we need to tailor the move function as well. I think the next goal is to create a new way to handle these data (change vectors within the states struct to prevent constant redefinition)

In [35]:
n_atoms = 55

ti = 550
tf = 900

n_traj = 20

temp = TempGrid{n_traj}(ti,tf)
mc_cycles = 1000
mc_sample = 1
displ_atom = 0.1 # Angstrom
n_adjust = 100
max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]
mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)
move_strat = MoveStrategy(atom_moves = n_atoms)  
ensemble = NVT(n_atoms)
bc_cu55 = SphericalBC(radius=14*AtoBohr)
start_config = Config(pos_cu55, bc_cu55)
n_bin = 100
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, suttonchenpot) for i in 1:n_traj]
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Matrix{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [20]:
ptmc_run!((mc_states, move_strat, mc_params, suttonchenpot, ensemble, results); save=false)
results.en_max

equilibration done


MC loop done.
cc


[0.20483691353034655, 0.2760021888033328, 0.20801943420650254, 0.16356330336131344, 0.18372807366721183, 0.3791318968348239, 0.20706622528531893, 0.29191311698733713, 0.5214723784030464, 0.33033394383028536, 0.35158502440772427, 0.5230247769862066, 0.3989315284645141, 0.2502862239262399, 0.2975674841214572, 0.6148424015042712, 0.5843053013104098, 0.5156293881617259, 0.29950202601760234, 0.5664572076342437]
done


-5.9117901173732434

In [36]:
mc_step!(mc_states,move_strat, mc_params, suttonchenpot, ensemble)

20-element Vector{MCState{Float64, 55, SphericalBC{Float64}}}:
 MCState{Float64, 55, SphericalBC{Float64}}(550.0, 574.1363368419949, Config{55, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[4.5112139495856e-6, -1.3138293967695e-5, 3.4601504321700004e-7], [-3.980885942754e-6, -3.3375035158614e-6, 4.575916827038472], [1.264752985534391, -3.8925157884546686, 2.0464230925690106], [-3.311163519470279, -2.405710469814843, 2.0464114164455425], [-3.311163596895427, 2.40569285113995, 2.0464011106786812], [1.264752859461583, 3.8924983184634674, 2.046406418069819], [4.092824837722801, -8.6887460654154e-6, 2.046420003785219], [-1.2647438377188622, -3.8925245957365804, -2.046405726039733], [-4.092815815294902, -1.75878418699746e-5, -2.0464193117551326], [-1.26474396379167, 3.8924895111815556, -2.046422400538924]  …  [2.0744963372369405, 6.384583971948036, -4.148977939714466], [6.5594487104586, 4.765688045720408, -4.053952548518178], [6.713184197946128, -2.1916113212552402e-5, -4.1489556556605

In [37]:
for state in mc_states
    println(state.en_tot)
end

-6.119268196146242
-6.119051293007541
-6.119270490671369
-6.119204307831756
-6.1192042863412865
-6.1192256828448555
-6.119246949010012
-6.119171007708147
-6.1191398396947045
-6.119125605267378
-6.1192289925321655
-6.119259155438917
-6.1190896181331755
-6.119169552433832
-6.119228727211807
-6.119195721385466
-6.119208389982615
-6.11918507470639
-6.119046128808509
-6.119264764524003


In [22]:
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, suttonchenpot) for i in 1:n_traj]
for state in mc_states
    println(state.en_tot)
end

-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012
-6.119246949010012


In [23]:
for i in 1:500
    mc_states = mc_cycle!(mc_states,move_strat,mc_params,suttonchenpot,ensemble,1,55,0,0)
    for state in mc_states
        if state.en_tot > 0 
            println("problem at $i")
        end
    end
end
for state in mc_states
    println(state.en_tot)
end

-6.097480333838161
-6.092999561405849
-6.094629180985897
-6.098957634301954
-6.097634600529146
-6.085922233980932
-6.096058156475707
-6.084104009392057
-6.088657425726878
-6.0971209273848395
-6.097220665481621
-6.095946330434252
-6.094696465914609
-6.090936435519146
-6.094491051629016
-6.0935157159499385
-6.089631176558095
-6.094195885497376
-6.096488060398768
-6.095726979889986


In [24]:
mc_states,results,delta_en_hist,delta_r2 = MCRun.equilibration_cycle!(mc_states,move_strat,mc_params,results,suttonchenpot,ensemble,1,55,0,0)

(MCState{Float64, 55, SphericalBC{Float64}}[MCState{Float64, 55, SphericalBC{Float64}}(550.0, 574.1363368419949, Config{55, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[0.13341605971141815, 0.04047302238802727, 0.09157149461047916], [-0.06362836470899921, -0.007222802079526698, 4.459129615868172], [1.3159316363493225, -3.9762042442221923, 2.125498115282241], [-3.442866286378093, -2.4178447945237003, 2.012442313547195], [-3.246372606247327, 2.3721658561471335, 2.018411675640692], [1.2043080140765101, 3.820699668943502, 1.9696256652338928], [4.124301847648825, -0.2495056695823281, 2.095394063486395], [-1.1999969843327758, -3.8522748160514615, -2.235496900047512], [-4.100750650400972, 0.01471523633442253, -2.1234703900624283], [-1.2917870915674805, 3.674496738791494, -2.075936511697765]  …  [2.0771876661340887, 6.435997860173391, -4.028971419253449], [6.526394699325196, 4.848748596196169, -3.98680183191805], [6.562541600018599, -0.07206887035895515, -4.282975670836168], [6.62755207

In [25]:
results.en_max

-6.073200956749535

In [65]:
pos_cu78 = [[9.20810577, 10.6811792, 12.85775222],
[7.72518083, 8.64747448, 13.47423093],
[8.83017234, 6.48185841, 14.26401845],
[5.76926904, 8.5204058, 11.78510919],
[10.3058934, 8.49163418, 13.72488251],
[11.0445983, 6.10374679, 13.03249449],
[10.76908039, 11.68540658, 11.04398002],
[6.86491181, 6.33673147, 12.63535963],
[11.14520009, 4.41652378, 11.06840909],
[10.56131156, 9.20976286, 11.36608831],
[3.8350787, 9.66043706, 10.62564335],
[8.18461754, 11.86457693, 10.78680942],
[12.53103163, 8.13651712, 12.41914688],
[9.10050302, 7.21655723, 11.96507541],
[8.9196861, 4.77149686, 12.37507079],
[8.03726493, 9.35893297, 11.12552957],
[11.75404842, 10.48292508, 13.05807496],
[7.19140363, 7.0877584, 10.30045396],
[6.63954366, 10.80307617, 12.52504858],
[13.20475821, 5.78540789, 11.69596475],
[10.04929714, 10.21021383, 2.74162567],
[8.60808677, 8.21325894, 3.29147489],
[11.13551339, 8.08991071, 3.51785384],
[7.18684228, 6.21910921, 3.93796442],
[7.88437782, 10.5718392, 3.95937956],
[9.71784555, 6.07918378, 4.09394596],
[11.97420313, 10.35137104, 4.33933113],
[6.42017871, 8.55796232, 4.53249281],
[9.97169751, 11.88885922, 4.62524758],
[9.82411407, 9.43691954, 5.10406749],
[5.02772356, 6.55318086, 5.20359556],
[5.71367449, 10.90921526, 5.25590498],
[8.41435724, 7.42843168, 5.68181186],
[11.0285349, 7.27653846, 5.85794355],
[7.79544252, 12.26033475, 5.87610665],
[4.27718147, 8.88880172, 5.86736713],
[11.91867483, 12.00967502, 6.24563146],
[6.9561607, 5.37238684, 6.26938662],
[7.67700429, 9.79132879, 6.32467407],
[9.50228326, 5.2069596, 6.45258385],
[11.87164769, 9.54494266, 6.68244807],
[12.02789873, 5.12581568, 6.74109416],
[9.88844903, 13.49922651, 6.57922127],
[3.59896936, 11.21261099, 6.58760866],
[6.25574475, 7.76131662, 6.94310978],
[9.77298468, 11.08697928, 7.00896014],
[5.64304702, 12.56406942, 7.20157052],
[9.65009283, 8.66311539, 7.41738673],
[12.95719784, 7.37463975, 7.57489662],
[3.97662306, 6.96385649, 7.4780558],
[5.5593229, 10.10255239, 7.6386882],
[7.73820851, 13.84622135, 7.8527848],
[8.1830049, 6.63840522, 8.0168064],
[10.70190431, 6.53159817, 8.22675048],
[13.72586678, 9.68714235, 8.3981167],
[7.98154103, 4.19765578, 8.32731731],
[3.27977734, 9.3022518, 8.18896381],
[7.62359206, 11.43808642, 8.27752581],
[10.55129337, 4.0805155, 8.5609616],
[11.76414234, 11.28499926, 8.65854603],
[5.91012759, 5.75557787, 8.56871289],
[7.49034308, 9.0057639, 8.68353281],
[11.54936709, 8.80691453, 9.05320905],
[9.7137393, 12.78636073, 8.95977573],
[12.67456914, 5.40926942, 9.22361494],
[4.12936341, 11.58348871, 9.01710543],
[5.22409251, 8.16647139, 9.25872611],
[9.56094879, 10.34182511, 9.36067818],
[9.37880896, 7.93821131, 9.68957149],
[6.19508081, 12.92482629, 9.64387612],
[13.5372612, 7.72382715, 10.06363071],
[4.95459032, 6.27620264, 10.88163222],
[9.20140742, 5.5571462, 10.04127471],
[6.0873292, 10.48531258, 10.10256552],
[6.96384327, 4.65312172, 10.66958722],
[9.02696245, 3.15942277, 10.40158812],
[11.28333689, 6.86978501, 10.68020117],
[12.79744324, 10.11163506, 10.75172547]]

cofm = [sum([pos[1] for pos in pos_cu78]),sum([pos[2] for pos in pos_cu78]),sum([pos[3] for pos in pos_cu78])]./78

for element in pos_cu78
    element .-=cofm 
end
println(pos_cu78)

[[0.47560863923076724, 2.097601591666667, 4.284427189615382], [-1.007316300769232, 0.06389687166666569, 4.900905899615381], [0.09767520923076844, -2.101719198333334, 5.690693419615382], [-2.963228090769232, -0.06317180833333325, 3.211784159615382], [1.5733962692307681, -0.0919434283333338, 5.1515574796153825], [2.3121011692307682, -2.479830818333334, 4.4591694596153815], [2.036583259230767, 3.1018289716666665, 2.4706549896153813], [-1.8675853207692326, -2.246846138333334, 4.062034599615382], [2.412702959230767, -4.167053828333334, 2.4950840596153814], [1.8288144292307678, 0.6261852516666657, 2.7927632796153823], [-4.897418430769232, 1.0768594516666656, 2.0523183196153827], [-0.5478795907692326, 3.280999321666666, 2.213484389615383], [3.798534499230767, -0.44706048833333334, 3.8458218496153815], [0.3680058892307674, -1.3670203783333337, 3.3917503796153827], [0.18718896923076755, -3.812080748333334, 3.8017457596153825], [-0.6952322007692331, 0.7753553616666657, 2.5522045396153814], [3.02

In [66]:
dismat78 = [distance2(a,b) for a in pos_cu78, b in pos_cu78]
println(get_energy(dismat78,suttonchenpot)[2])
energytotal(dismat78,suttonchenpot)

1159.4240958269834


1159.4240958269834